In [12]:
import asyncio, boto3, pandas as pd, os, sys, pprint
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

DAILY_UPLOADS_TABLE = "DailyUploadsTable-dev"
ddb = boto3.client("dynamodb", region_name="ap-south-1")

In [140]:
# import asyncio, concurrent.futures, functools
# from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool
from subprocess import Popen, PIPE
from time import time

class ProcessPosts:
    def __init__(self, subreddit, posts,logger):
        self.subreddit=subreddit
        self.posts = posts
        self.download_cmd = [
            'youtube-dl',
            '-f',
            'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
            'placeholder_url',
            '-o',
            'placeholder_title'
        ]
        self.download_processes = []
        self.logger = logger
        self.download_processes_logs = []
        self.tasks = {}
        self.logger.info('Initialized object')
        
    
    def download_videos_from_posts(self):      
        start = time()
        with Pool(len(self.posts)) as p:
            p.map(download_video_from_post, self.posts)
        end = time()
        
        self.logger.info(f"Time taken to download {len(self.posts)} videos from {self.subreddit} is {round(end-start, 2)}s")


def download_video_from_post(post):
    logger = logging.getLogger()
    download_cmd = [
            'youtube-dl',
            '-f',
            'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
            'placeholder_url',
            '-o',
            'placeholder_title'
    ]

    download_cmd[3] = post['url']
    download_cmd[5] = post['title']       


    process = Popen(download_cmd, stderr=PIPE)
    stdout, stderr = process.communicate()

    if stderr:
        logger.error("Failed to download the post:\n")
        logger.error(pp.pformat(post))
        logger.error("Failed with error")
        logger.error(pp.pformat(stderr.decode('utf-8')))   
    else:
        logger.info("Successfully downloaded the post:\n")
        logger.info(pp.pformat(post))
        

# youtube-dl  command

In [141]:
# !  youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best' https://www.reddit.com/r/IdiotsInCars/comments/p4cm -o title

# !  youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best' https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/ -o title


In [142]:
import logging
posts = [
    {
        'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/',
        'title': 'I think this belongs here'
    },
    {
        'url': 'https://www.reddit.com/r/IdiotsInCars/commen',
        'title': 'I there'
    },
   
]

# Initialize log config.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [143]:
for idx, post in enumerate(posts):
    print(idx, post)

0 {'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/', 'title': 'I think this belongs here'}
1 {'url': 'https://www.reddit.com/r/IdiotsInCars/commen', 'title': 'I there'}


In [144]:
process_posts = ProcessPosts(subreddit='funny', posts=posts, logger=logger)

INFO:root:Initialized object


In [156]:
title = '  I think this belongs here. '

title = title.strip()

# title = title if title[:-1] != '.' else title[:-1]

title

'I think this belongs here.'

In [151]:
process_posts.download_videos_from_posts()

[RedditR] p4cmjt: Downloading JSON metadata
[generic] commen: Requesting header
[generic] commen: Downloading webpage


ERROR:root:Failed to download the post:

ERROR:root:{'title': 'I there', 'url': 'https://www.reddit.com/r/IdiotsInCars/commen'}
ERROR:root:Failed with error
ERROR:root:('WARNING: Could not send HEAD request to '
 'https://www.reddit.com/r/IdiotsInCars/commen: HTTP Error 404: Not Found\n'
 'ERROR: Unable to download webpage: HTTP Error 404: Not Found (caused by '
 "<HTTPError 404: 'Not Found'>); please report this issue on "
 'https://yt-dl.org/bug . Make sure you are using the latest version; see  '
 'https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with '
 'the --verbose flag and include its complete output.\n')


[Reddit] nj6jgivg2dh71: Downloading m3u8 information
[Reddit] nj6jgivg2dh71: Downloading MPD manifest
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 5
[download] Destination: I think this belongs here.fhls-1530
[download] 100% of 4.86MiB in 00:01.72MiB/s ETA 00:00:05
[download] Destination: I think this belongs here.fdash-audio_0_132208
[download]  52.8% of 482.72KiB at  3.86MiB/s ETA 00:00known ETA

INFO:root:Successfully downloaded the post:

INFO:root:{ 'title': 'I think this belongs here',
  'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/'}
INFO:root:Time taken to download 2 videos from funny is 4.13s


[download] 100% of 482.72KiB in 00:0057MiB/s ETA 00:00
[ffmpeg] Merging formats into "I think this belongs here.mp4"
Deleting original file I think this belongs here.fhls-1530 (pass -k to keep)
Deleting original file I think this belongs here.fdash-audio_0_132208 (pass -k to keep)


[RedditR] p4cmjt: Downloading JSON metadata
[RedditR] p4cmjt: Downloading JSON metadata
[Reddit] nj6jgivg2dh71: Downloading m3u8 information
[Reddit] nj6jgivg2dh71: Downloading MPD manifest
[hlsnative] Downloading m3u8 manifest
[Reddit] nj6jgivg2dh71: Downloading m3u8 information
[hlsnative] Total fragments: 5
[download] Destination: I there.fhls-1530
[Reddit] nj6jgivg2dh71: Downloading MPD manifest
[download]  10.2% of ~4.89MiB at  4.07MiB/s ETA 00:01[hlsnative] Downloading m3u8 manifest
[download]  20.0% of ~4.89MiB at  4.77MiB/s ETA 00:01[hlsnative] Total fragments: 5
[download] Destination: I think this belongs here.fhls-1530
[download] 100% of 4.86MiB in 00:06.09MiB/s ETA 00:00851
[download] Destination: I there.fdash-audio_0_132208
[download] 100% of 482.72KiB in 00:0015MiB/s ETA 00:00known ETA
[ffmpeg] Merging formats into "I there.mp4"
Deleting original file I there.fhls-1530 (pass -k to keep)
Deleting original file I there.fdash-audio_0_132208 (pass -k to keep)
[download]  20.